Utilizando VGG16 para gerar embeddings

In [ ]:
import os
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

#Carregar o modelo pré-treinado
model = torch.hub.load('pytorch/vision:v0.9.0', 'vgg16', pretrained=True)
model = model.to('cuda')
model.eval()

#Transformações para as imagens
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

#Diretório das imagens
image_dir = r"C:\Users\heloi\Downloads\yelp_photos\photos"

#Loop para extrair embeddings de todas as imagens do diretório
embeddings = []
for filename in tqdm(os.listdir(image_dir)):
    if filename.endswith(".jpg"):
        try:
            filepath = os.path.join(image_dir, filename)

            # Carregar e transformar a imagem
            image = Image.open(filepath).convert("RGB")
            image_tensor = transform(image).unsqueeze(0)
            image_tensor = image_tensor.to('cuda')
            
            # Obter o embedding da imagem
            with torch.no_grad():
                embedding = model(image_tensor).squeeze().cpu().numpy()
                embeddings.append(embedding)
        except:
            print(f"Imagem {filename} não pode ser lida, foi descartada.")
#Salvar embeddings em um arquivo npy
embeddings = np.array(embeddings)
np.save("embeddingsimagens.npy", embeddings)


In [4]:
import numpy as np

# Carrega o arquivo de embeddings
embeddings = np.load('embeddingsimagens.npy')

# Verifica as dimensões dos embeddings
print(embeddings.shape)


(199992, 1000)


É possível verificar que de todas as 200098 imagens, geramos apenas 199992, visto que algumas imagens estavam corrompidas.